In [2]:
from lxml import etree
import pandas as pd
import requests # for http requests

def get_commodity_from_investing(commodity_name):
    url = "https://www.investing.com/commodities/%s-historical-data"%commodity_name
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}) # add User-Agent to prevent 403 error
    html = response.content
    tree = etree.HTML(html)

    rows = len(tree.xpath('/html/body/div[5]/section/div[9]/table[1]/tbody/tr[*]')) # use * to get all rows
    price = [0]*rows
    date = ['']*rows
    for i in range(rows):
        date[i] = tree.xpath('/html/body/div[5]/section/div[9]/table[1]/tbody/tr[%s]/td[1]/text()'%str(i+1))[0] # index 0 to remove the list item
        price[i] = tree.xpath('/html/body/div[5]/section/div[9]/table[1]/tbody/tr[%s]/td[2]/text()'%str(i+1))[0]
    #gold_date_price = list(zip(date,price)) # zip the two lists into one
    return date,price
def date_formatting(date):
    import time
gold_date,gold_price = get_commodity_from_investing('gold')
silver_date,silver_price = get_commodity_from_investing('silver')

#data = pd.DataFrame({'Date':gold_date,'Gold_price':gold_price,'Silver_price':silver_price})

ValueError: arrays must all be same length

In [5]:
print(gold_date,gold_price)
print(silver_date,silver_price)

['Dec 07, 2018', 'Dec 06, 2018', 'Dec 05, 2018', 'Dec 04, 2018', 'Dec 03, 2018', 'Nov 30, 2018', 'Nov 29, 2018', 'Nov 28, 2018', 'Nov 27, 2018', 'Nov 26, 2018', 'Nov 23, 2018', 'Nov 22, 2018', 'Nov 21, 2018', 'Nov 20, 2018', 'Nov 19, 2018', 'Nov 16, 2018', 'Nov 15, 2018', 'Nov 14, 2018', 'Nov 13, 2018', 'Nov 12, 2018', 'Nov 09, 2018', 'Nov 08, 2018'] ['1,252.60', '1,243.60', '1,242.60', '1,246.60', '1,239.60', '1,226.00', '1,230.40', '1,229.80', '1,219.90', '1,228.70', '1,229.10', '1,228.00', '1,233.80', '1,227.00', '1,231.10', '1,229.00', '1,221.10', '1,216.00', '1,207.30', '1,203.50', '1,208.60', '1,225.10']
['Dec 07, 2018', 'Dec 06, 2018', 'Dec 05, 2018', 'Dec 04, 2018', 'Dec 03, 2018', 'Dec 02, 2018', 'Nov 30, 2018', 'Nov 29, 2018', 'Nov 28, 2018', 'Nov 27, 2018', 'Nov 26, 2018', 'Nov 25, 2018', 'Nov 23, 2018', 'Nov 22, 2018', 'Nov 21, 2018', 'Nov 20, 2018', 'Nov 19, 2018', 'Nov 18, 2018', 'Nov 16, 2018', 'Nov 15, 2018', 'Nov 14, 2018', 'Nov 13, 2018', 'Nov 12, 2018', 'Nov 11, 2018